In [44]:
import io
import random
import string
import warnings
import numpy as np
import tkinter as tk
from tkinter import scrolledtext
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem import WordNetLemmatizer

# Suppress warnings
warnings.filterwarnings('ignore')

# Download necessary NLTK data
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)

# Load admission-related corpus
with open('C:\\Users\\Asus\\Desktop\\SEM-8\\FDTL.DATA SCIENCE\\bots\\admission_bot_large.txt', 'r', encoding='utf8', errors='ignore') as fin:
    raw = fin.read().lower()

# Tokenization
sent_tokens = nltk.sent_tokenize(raw)

# Preprocessing
lemmer = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = str.maketrans('', '', string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# Greeting Matching
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up", "hey")
GREETING_RESPONSES = [
    "🙏 Jay Shree Krishna!!!",  
    "🛕 Namaste :)",  
    "🤝 Ram Ram!!! How can I assist you with admissions?",  
    "👋 Hi there ;)",  
    "🕉️ Radhe Radhe!!! Ask me anything about admissions"
]

def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)
    return None

# FAQs
FAQS = {
    "eligibility criteria": "The eligibility criteria depend on the course. Typically, you need a minimum of 50% in your previous qualifying exam.",
    "admission process": "You can apply online on the university website. Make sure to submit your documents and pay the application fee.",
    "scholarships available": "Yes, we offer merit-based and need-based scholarships. You can check the university website for more details.",
    "contact information": "You can reach out to us at info@ljku.edu.in or call 18001210082.",
    "application deadline": "Deadlines vary by course. Please check the official university website for up-to-date information."
}

# Generate chatbot response
def get_response(user_response):
    user_response = user_response.lower().strip()

    if user_response in FAQS:
        return FAQS[user_response]

    sent_tokens.append(user_response)

    TfidfVec = TfidfVectorizer(stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)

    vals = cosine_similarity(tfidf[-1], tfidf[:-1])
    scores = vals.flatten()
    related_indices = scores.argsort()[-2:][::-1]

    responses = [sent_tokens[i] for i in related_indices if scores[i] > 0.1]
    sent_tokens.pop()

    if not responses:
        return "I'm sorry! I don't have info on that. Try asking about admissions, eligibility, scholarships, or deadlines."

    return "\n".join([f"➡ {resp}" for resp in responses])

# GUI with Tkinter
# --- inside class ChatBotGUI ---
class ChatBotGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("🎓 LJ University Admission ChatBot")
        self.root.geometry("600x500")
        self.root.resizable(False, False)
        self.root.configure(bg="#2c3e50")  # Dark background for a modern feel

        # --- Title Banner ---
        self.header = tk.Label(root, text="🎓 LJ University Admission ChatBot", font=("Helvetica", 18, "bold"), bg="#34495e", fg="#ecf0f1", pady=10)
        self.header.pack(fill=tk.X)

        # --- Chat Display Area ---
        self.chat_area = scrolledtext.ScrolledText(
            root,
            wrap=tk.WORD,
            state='disabled',
            font=("Segoe UI", 11),
            bg="#ecf0f1",
            fg="#2c3e50",
            bd=0,
            padx=10,
            pady=10
        )
        self.chat_area.place(x=10, y=60, width=580, height=340)

        # --- User Entry Field ---
        self.entry = tk.Entry(
            root,
            font=("Segoe UI", 12),
            bg="#ffffff",
            fg="#000000",
            bd=2,
            relief=tk.FLAT,
        )
        self.entry.place(x=10, y=420, width=480, height=35)
        self.entry.bind("<Return>", self.send_message)

        # --- Send Button ---
        self.send_btn = tk.Button(
            root,
            text="Send",
            font=("Segoe UI", 11, "bold"),
            bg="#1abc9c",
            fg="white",
            activebackground="#16a085",
            activeforeground="white",
            bd=0,
            command=self.send_message,
            cursor="hand2"
        )
        self.send_btn.place(x=500, y=420, width=90, height=35)

        # First welcome message
        self.display_message("🤖", "Welcome! Ask me anything about university admissions.")

    def display_message(self, sender, message):
        self.chat_area.configure(state='normal')
        self.chat_area.insert(tk.END, f"{sender}: {message}\n\n")
        self.chat_area.configure(state='disabled')
        self.chat_area.see(tk.END)

    def send_message(self, event=None):
        user_msg = self.entry.get().strip()
        if user_msg == "":
            return

        self.display_message("👤 You", user_msg)
        self.entry.delete(0, tk.END)

        if user_msg.lower() in ['bye', 'exit', 'quit']:
            self.display_message("🤖", "Goodbye! Let me know if you need more help.")
            self.root.after(2000, self.root.destroy)
            return
        elif user_msg.lower() in ['thanks', 'thank you']:
            self.display_message("🤖", "You're welcome! Good luck with your application. 🎓")
            return

        greet = greeting(user_msg)
        bot_reply = greet if greet else get_response(user_msg)
        self.display_message("🤖", bot_reply)


# Run the GUI
if __name__ == "__main__":
    root = tk.Tk()
    gui = ChatBotGUI(root)
    root.mainloop()


In [5]:
# !pip install textblob fuzzywuzzy googletrans
!pip install SpeechRecognition


   ---------------------------------------- 0.0/32.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/32.9 MB 1.3 MB/s eta 0:00:26
   ---------------------------------------- 0.1/32.9 MB 1.1 MB/s eta 0:00:31
   ---------------------------------------- 0.2/32.9 MB 1.5 MB/s eta 0:00:23
   ---------------------------------------- 0.2/32.9 MB 1.3 MB/s eta 0:00:27
   ---------------------------------------- 0.3/32.9 MB 1.3 MB/s eta 0:00:26
   ---------------------------------------- 0.4/32.9 MB 1.3 MB/s eta 0:00:25
   ---------------------------------------- 0.4/32.9 MB 1.3 MB/s eta 0:00:26
    --------------------------------------- 0.5/32.9 MB 1.4 MB/s eta 0:00:24
    --------------------------------------- 0.5/32.9 MB 1.4 MB/s eta 0:00:24
    --------------------------------------- 0.6/32.9 MB 1.3 MB/s eta 0:00:25
    --------------------------------------- 0.7/32.9 MB 1.3 MB/s eta 0:00:25
    --------------------------------------- 0.7/32.9 MB 1.3 MB/s eta 0:00:25
    --

In [50]:
# Full-featured University Admission ChatBot with GUI (Tkinter)

import tkinter as tk
from tkinter import scrolledtext
import io
import random
import string
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem import WordNetLemmatizer
from termcolor import colored
from pyfiglet import figlet_format
from textblob import TextBlob
from fuzzywuzzy import process
from googletrans import Translator
import speech_recognition as sr

warnings.filterwarnings('ignore')

# Download required NLTK data
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)

# Load corpus
with open('C:\\Users\\Asus\\Desktop\\SEM-8\\FDTL.DATA SCIENCE\\bots\\admission_bot_large.txt', 'r', encoding='utf8', errors='ignore') as fin:
    raw = fin.read().lower()

sent_tokens = nltk.sent_tokenize(raw)
lemmer = WordNetLemmatizer()
translator = Translator()
last_question = ""

FAQS = {
    "eligibility criteria": "The eligibility criteria depend on the course. Typically, you need a minimum of 50% in your previous qualifying exam.",
    "admission process": "You can apply online on the university website. Make sure to submit your documents and pay the application fee.",
    "scholarships available": "Yes, we offer merit-based and need-based scholarships.",
    "contact information": "Reach us at info@ljku.edu.in or call 18001210082.",
    "application deadline": "Deadlines vary. Please check the official website."
}

GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up", "hey")
GREETING_RESPONSES = [
    "🙏 Jay Shree Krishna!!!",  
    "🛕 Namaste :)",  
    "🤝 Ram Ram!!! How can I assist you with admissions?",  
    "👋 Hi there ;)",  
    "🕉️ Radhe Radhe!!! Ask me anything about admissions"
]

def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)
    return None

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

def LemNormalize(text):
    remove_punct_dict = str.maketrans('', '', string.punctuation)
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

def summarize_response(text):
    blob = TextBlob(text)
    sentences = blob.sentences
    if len(sentences) > 2:
        return str(sentences[0]) + " " + str(sentences[1])
    return text

def fuzzy_faq_match(user_input):
    best_match, score = process.extractOne(user_input, FAQS.keys())
    if score > 75:
        return FAQS[best_match]
    return None

def translate_input(text):
    try:
        result = translator.translate(text, dest='en')
        return result.text
    except:
        return text

def get_response(user_input):
    global last_question
    user_input = translate_input(user_input)
    last_question = user_input

    if user_input.lower() in FAQS:
        return FAQS[user_input.lower()]

    fuzzy_result = fuzzy_faq_match(user_input)
    if fuzzy_result:
        return fuzzy_result

    sent_tokens.append(user_input)
    TfidfVec = TfidfVectorizer(stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf[:-1])
    scores = vals.flatten()
    related_indices = scores.argsort()[-2:][::-1]
    responses = [sent_tokens[i] for i in related_indices if scores[i] > 0.1]
    sent_tokens.pop()

    if not responses:
        return "I’m not sure about that. You can ask about: eligibility, deadlines, scholarships, or the admission process."

    return summarize_response(" ".join(responses))

# --- GUI with Tkinter ---
class ChatBotGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("🎓 LJ University Admission ChatBot")
        self.root.geometry("600x500")
        self.root.resizable(False, False)
        self.root.configure(bg="#2c3e50")

        self.header = tk.Label(root, text="🎓 LJ University Admission ChatBot", font=("Helvetica", 18, "bold"), bg="#34495e", fg="#ecf0f1", pady=10)
        self.header.pack(fill=tk.X)

        self.chat_area = scrolledtext.ScrolledText(root, wrap=tk.WORD, state='disabled', font=("Segoe UI", 11), bg="#ecf0f1", fg="#2c3e50", bd=0, padx=10, pady=10)
        self.chat_area.place(x=10, y=60, width=580, height=340)

        self.entry = tk.Entry(root, font=("Segoe UI", 12), bg="#ffffff", fg="#000000", bd=2, relief=tk.FLAT)
        self.entry.place(x=10, y=420, width=480, height=35)
        self.entry.bind("<Return>", self.send_message)

        self.send_btn = tk.Button(root, text="Send", font=("Segoe UI", 11, "bold"), bg="#1abc9c", fg="white", activebackground="#16a085", activeforeground="white", bd=0, command=self.send_message, cursor="hand2")
        self.send_btn.place(x=500, y=420, width=90, height=35)

        self.display_message("🤖", "Welcome! Ask me anything about university admissions.")

    def display_message(self, sender, message):
        self.chat_area.configure(state='normal')
        self.chat_area.insert(tk.END, f"{sender}: {message}\n\n")
        self.chat_area.configure(state='disabled')
        self.chat_area.see(tk.END)

    def send_message(self, event=None):
        user_msg = self.entry.get().strip()
        if user_msg == "":
            return

        self.display_message("👤 You", user_msg)
        self.entry.delete(0, tk.END)

        if user_msg.lower() in ['bye', 'exit', 'quit']:
            self.display_message("🤖", "Goodbye! Let me know if you need more help.")
            self.root.after(2000, self.root.destroy)
            return
        elif user_msg.lower() in ['thanks', 'thank you']:
            self.display_message("🤖", "You're welcome! Good luck with your application. 🎓")
            return

        greet = greeting(user_msg)
        bot_reply = greet if greet else get_response(user_msg)
        self.display_message("🤖", bot_reply)

# Entry point
if __name__ == "__main__":
    root = tk.Tk()
    app = ChatBotGUI(root)
    root.mainloop()